In [180]:
import torch
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
from torch.autograd import Variable
from torch.nn.functional import relu

In [2]:
def read_data():
    with open('D:\\BDT lecture notes\\5008(Social Computing)\\fb\\facebook_combined.txt','rb') as f:
        data = f.readlines()
    
    data = [bytes.decode(r).strip('\n') for r in data]
    data = [r.split(' ') for r in data]
    data = [[int(r[0]),int(r[1])] for r in data]
    data = [tuple(pair) for pair in data]
    
    return data

In [3]:
def produce_adj(data):
    all_node = []
    for pair in data:
        all_node.append(pair[0])
        all_node.append(pair[1])
    all_node = list(set(all_node))
    adj = np.zeros([len(all_node),len(all_node)])
    for pair in data:
        adj[pair[0],pair[1]] = 1
        adj[pair[1],pair[0]] = 1
    return adj

# 读取数据

In [4]:
data= read_data()
adj = produce_adj(data)  #包含全部数据的邻接矩阵

In [5]:
ix = np.array(list(range(adj.shape[0])))     

# 选取了列编号为1888的用户，其所有粉丝张成一个community

因为他粉丝最多，有需要也可以选其他人

In [181]:
l = list(ix * adj[:,1888])
l = [int(r) for r in l if r!=0]

In [182]:
community = adj[np.ix_(l,l)]

#community是这个圈子构成的社交网络的adjacency matrix

In [185]:
Community = 2/math.pi*torch.atan(relu(Variable(torch.tensor(100 * community),requires_grad=True)))

Ones_1n = torch.ones([1,Community.shape[0]]).type(torch.DoubleTensor)   #方便起见，先写好矩阵对应的1向量，常用
Ones_n1 = torch.ones([Community.shape[0],1]).type(torch.DoubleTensor)

In [186]:
Community

tensor([[0.0000, 0.9936, 0.9936,  ..., 0.9936, 0.9936, 0.9936],
        [0.9936, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.9936, 0.0000, 0.0000,  ..., 0.9936, 0.0000, 0.0000],
        ...,
        [0.9936, 0.0000, 0.9936,  ..., 0.0000, 0.0000, 0.0000],
        [0.9936, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9936],
        [0.9936, 0.0000, 0.0000,  ..., 0.0000, 0.9936, 0.0000]],
       dtype=torch.float64, grad_fn=<MulBackward0>)

# 七个统计量

In [112]:
def Skewness(series):
    mean = torch.mean(series)
    diffs = Degree - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    skews = torch.mean(torch.pow(zscores, 3.0))
    return skews

def Kurtoses(series):
    mean = torch.mean(series)
    diffs = Degree - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0
    return kurtoses

def Canberra(y,y_pred):
    return torch.sum(torch.abs(y.view_as(y_pred) - y_pred) / (y_pred + y.view_as(y_pred)))

In [9]:
#Clustering coefficient
triangle = (np.matmul(community,community)*community)
triangle = np.matmul(triangle,np.ones([triangle.shape[0],1])).T/2
degree = np.matmul(np.ones([1,community.shape[0]]),community)# + np.matmul(community,np.ones([community.shape[0],1])).T
degree_2 = degree * (degree-1)
clust_coef = 2*triangle/degree_2

In [10]:
#Clustering coefficient(Torch)
Triangle = (Community.mm(Community)).mul(Community)
Triangle = Triangle.mm(Ones_n1).T/2
Degree = Ones_1n.mm(Community)
Degree_2 = Degree.mul(Degree-1)
Clust_coef = 2 * Triangle/Degree_2

In [11]:
#Degrees
degree = np.matmul(np.ones([1,community.shape[0]]),community)

In [12]:
#Degree(Torch)
Degree = Ones_1n.mm(Community)

In [13]:
#average degree of neibourhood
degree = np.matmul(np.ones([1,community.shape[0]]),community)
neibour_degree = np.matmul(degree,community) * np.ones([1,community.shape[0]])/degree

In [14]:
#average degree of neibourhood(Torch)
Degree = Ones_1n.mm(Community)
Neibour_degree = (Degree.mm(Community)).mul(Ones_1n)/Degree

In [15]:
#average clustering coef of neibourhood
triangle = (np.matmul(community,community)*community)
triangle = np.matmul(triangle,np.ones([triangle.shape[0],1])).T/2
degree = np.matmul(np.ones([1,community.shape[0]]),community)# + np.matmul(community,np.ones([community.shape[0],1])).T
degree_2 = degree * (degree-1)
clust_coef = 2*triangle/degree_2
neibour_clust_coef = np.matmul(clust_coef,community)
neibour_aver_clust_coef = neibour_clust_coef * np.ones([1,community.shape[0]])/degree

In [16]:
#average clustering coef of neibourhood(Torch)
Triangle = (Community.mm(Community)).mul(Community)
Triangle = Triangle.mm(Ones_n1).T/2
Degree = Ones_1n.mm(Community)
Degree_2 = Degree.mul(Degree-1)
Clust_coef = 2 * Triangle/Degree_2
Neibour_clust_coef = Clust_coef.mm(Community)
Neibour_aver_clust_coef = Neibour_clust_coef.mul(Ones_1n)/Degree

In [17]:
# number of edges in the neighborhood
edge_ego = np.zeros([1,community.shape[0]])
for i in range(community.shape[0]):
    t = np.matmul(np.matmul(np.ones([1,community.shape[0]]),community[[i],:] * community * community[:,[i]]),np.ones([community.shape[0],1]))/community[[i],:].sum()
    edge_ego[0,i] = t

In [31]:
# number of edges in the neighborhood(Torch)
Edge_ego = torch.zeros([1,Community.shape[0]])
for i in range(community.shape[0]):
    Edge_ego[0,i] = Ones_1n.mm(Community[[i],:].mul(Community).mul(Community[:,[i]])).mm(Ones_n1)/Community[[i],:].sum()

In [65]:
#number of outgoing edges from the neighborhood
mid = community.copy()
mid+=np.diag(np.ones([community.shape[0]]))
out_edge_neibour = np.zeros([1,community.shape[0]])
for i in range(mid.shape[0]):
    out_edge_neibour[0,i] = np.diag(mid - mid[[i],:] * mid * mid[:,[i]]).sum()

In [62]:
#number of outgoing edges from the neighborhood(Torch)
mid_1 = Community + torch.diag(torch.ones([1,Community.shape[0]]).squeeze())
Out_edge_neibour = torch.zeros([1,Community.shape[0]])
for i in range(mid_1.shape[0]):
    Out_edge_neibour[0,i] = torch.diag(mid_1 - mid_1[[i],:].mul(mid_1).mul(mid_1[:,[i]])).sum()

In [85]:
# number of neighbors of neighbors (not in neighborhood)
mid = community.copy()
mid+=np.diag(np.ones([mid.shape[0]]))
neibour_neibour = np.zeros([1,community.shape[0]])
for i in range(mid.shape[0]):
    neibour = mid - mid[[i],:] * mid * mid[:,[i]]
    d = np.diag(neibour)
    neibour_neibour[0,i]=np.sum(d.reshape(1,-1)*neibour*d.reshape(-1,1))

In [82]:
# number of neighbors of neighbors (not in neighborhood)(Torch)
mid_2 = Community+torch.diag(torch.ones([1,Community.shape[0]]).squeeze())
Neibour_neibour = torch.zeros([1,Community.shape[0]])
for i in range(mid.shape[0]):
    Neibour_neibour[0,i] = ((torch.diag(mid_2 - mid_2[[i],:].mul(mid_2).mul(mid_2[:,[i]])).reshape(1,-1)).mul(mid_2 - mid_2[[i],:].mul(mid_2).mul(mid_2[:,[i]])).mul(torch.diag(mid_2 - mid_2[[i],:].mul(mid_2).mul(mid_2[:,[i]])).reshape(-1,1))).sum()

In [113]:
Vars = [Degree,Clust_coef,Neibour_degree,Neibour_aver_clust_coef,Edge_ego,Out_edge_neibour,Neibour_neibour]

In [187]:
Netsimile = torch.zeros(7,4)
for i in range(7):
    Netsimile[i,0] = Vars[i].mean()
    Netsimile[i,1] = Vars[i].std()
    Netsimile[i,2] = 1/Skewness(Vars[i])
    Netsimile[i,3] = 1/Kurtoses(Vars[i])
    
Netsimile = Netsimile.reshape(-1,1)

# 随机去除边

In [239]:
index = []

list_x = list(np.where(community == 1)[0])
list_y = list(np.where(community == 1)[1])


In [240]:
l

[12, 34, 5, 6, 7, 88, 9]